## テストデータを1日ずつ予測するための検証

In [3]:
import gc
import os
import time
import pickle
import warnings
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from typing import Union
import matplotlib.pyplot as plt
from sklearn import preprocessing, metrics
from tqdm import tqdm
from scipy.sparse import csr_matrix

warnings.filterwarnings('ignore')
"""
TODO 365 + 28日分をtrainデータとしてlag特徴量を作成し、古い28日分は欠損値が出るので削除

"""

# メモリ使用量の削減
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


# function to read the data and merge it
# (ignoring some columns, this is a very fst model)
def read_data():
    print('Reading files...')

    calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
    calendar_df = reduce_mem_usage(calendar_df)
    print('Calendar: ' + str(calendar_df.shape))

    sell_prices_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
    sell_prices_df = reduce_mem_usage(sell_prices_df)
    print('Sell prices: ' + str(sell_prices_df.shape))

    train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
    print('Sales train validation: ' + str(train_df.shape))

    submission_df = pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv')
    print("Submission: " + str(submission_df.shape))

    return calendar_df, sell_prices_df, train_df, submission_df


def melt_and_merge(calendar_df, sell_prices_df, train_df, submission_df, num_train_data):

    # trainは直近１年間のデータのみ使用
    drop_columns = [f"d_{d}" for d in range(1, (1913 - num_train_data) + 1)]
    train_df.drop(drop_columns, inplace = True, axis=1)
    print("\ntrainは直近１年間のデータのみ使用")
    print('Sales train validation(remain only one year): ' + str(train_df.shape))

    # 商品情報を抽出
    product_df = train_df.loc[:, "id":"state_id"]

    # 列方向に連なっていたのを変形し行方向に連ねるように整理
    train_df = pd.melt(train_df, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                       var_name='day', value_name='demand')

    train_day = train_df["day"].unique()
    print("train_data: {0} ~ {1} -> {2}".format(train_day[0], train_day[-1], len(train_day)))

    # seperate test dataframes
    stage1_eval_df = submission_df[submission_df["id"].str.contains("validation")]
    stage2_eval_df = submission_df[submission_df["id"].str.contains("evaluation")]

    # change column names
    stage1_eval_df.columns = ["id"] + [f"d_{d}" for d in range(1914, 1942)]  # F1 ~ F28 => d_1914 ~ d_1941
    stage2_eval_df.columns = ["id"] + [f"d_{d}" for d in range(1942, 1970)]  # F1 ~ F28 => d_1942 ~ d_1969

    # melt, mergeを使ってsubmission用のdataframeを上のsales_train_validationと同様の形式に変形
    stage1_eval_df = stage1_eval_df.merge(product_df, how='left', on='id')
    stage1_eval_df = pd.melt(stage1_eval_df, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                             var_name='day', value_name='demand')
    stage1_day = stage1_eval_df["day"].unique()
    print("[STAGE1] eval_data: {0} ~ {1} -> {2}".format(stage1_day[0], stage1_day[-1], len(stage1_day)))

    # train_df, stage1_eval_dfと同様にstage2_eval_dfとproduct_dfをmergeさせたい
    # しかしidが_evaluationのままだとデータが一致せずmergeできないので一時的に_validationにidを変更
    stage2_eval_df['id'] = stage2_eval_df.loc[:, 'id'].str.replace('_evaluation', '_validation')
    stage2_eval_df = stage2_eval_df.merge(product_df, how='left', on='id')
    stage2_eval_df['id'] = stage2_eval_df.loc[:, 'id'].str.replace('_validation', '_evaluation')
    stage2_eval_df = pd.melt(stage2_eval_df, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                             var_name='day', value_name='demand')
    stage2_day = stage2_eval_df["day"].unique()
    print("[STAGE2] eval_data: {0} ~ {1} -> {2}".format(stage2_day[0], stage2_day[-1], len(stage2_day)))

    train_df['part'] = 'train'
    stage1_eval_df['part'] = 'stage1'
    stage2_eval_df['part'] = 'stage2'

    data_df = pd.concat([train_df, stage1_eval_df, stage2_eval_df], axis=0)
    data_df = reduce_mem_usage(data_df)
    # print("\n[INFO] data_df(after merge valid & eval) ->")
    # data_df.head()

    # 不要なdataframeの削除
    del train_df, stage1_eval_df, stage2_eval_df, product_df

    # drop some calendar features
    calendar_df.drop(['weekday', 'wday', 'month', 'year'], inplace=True, axis=1)

    # delete stage2_eval_df for now
    data_df = data_df[data_df['part'] != 'stage2']
    print("[CHECK] Remove the stage2 eval data")

    # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
    data_df = pd.merge(data_df, calendar_df, how='left', left_on=['day'], right_on=['d'])
    data_df.drop('d', inplace=True, axis=1)

    # get the sell price data (this feature should be very important)
    data_df = data_df.merge(sell_prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    # print("\n[INFO] data_df(after merge calendar & prices) ->")
    # print(data_df.head(5))
    # print(data_df.columns)

    return data_df


# label encoding
def encode_categorical(data_df):
    nan_features = ['event_name_1', 'event_type_1',
                    'event_name_2', 'event_type_2']
    for feature in nan_features:
        # label encodingのためnanを文字列に変換
        data_df[feature].fillna('unknown', inplace=True)

    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
           'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data_df[feature] = encoder.fit_transform(data_df[feature])

    return data_df


def data_split(data_df):
    # train data
    train_df = data_df[data_df['part'] == 'train']

    # stage1 eval data(特徴量生成用に56日分多めに確保しておく)
    test_df = data_df[(data_df['day'] > 'd_1857')]  # 56日前~(lag特徴量生成に使用) -> 1914(予測対象) ~

    del data_df
    gc.collect()

    return train_df, test_df


def feature_engineering(data_df):
    """
    1日後のリード特徴量
    1日前のラグ特徴量
    """

    # ラグ特徴量
    data_df['lag7'] = data_df.groupby(['id'])['demand'].shift(7)
    data_df['lag28'] = data_df.groupby(['id'])['demand'].shift(28)
    data_df['rmean_lag7_7'] = data_df.groupby(['id'])['lag7'].transform(lambda x: x.rolling(7).mean())
    data_df['rmean_lag7_28'] = data_df.groupby(['id'])['lag7'].transform(lambda x: x.rolling(28).mean())
    data_df['rmean_lag28_7'] = data_df.groupby(['id'])['lag28'].transform(lambda x: x.rolling(7).mean())
    data_df['rmean_lag28_28'] = data_df.groupby(['id'])['lag28'].transform(lambda x: x.rolling(28).mean())

    # price features
    # data_df['sell_price_lag1'] = data_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    # data_df['sell_price_lag7'] = data_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(7))
    # data_df['sell_price_lag28'] = data_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(28))
    # mean_sell_price_df = data_df.groupby('id').mean()
    # mean_sell_price_df.rename(columns={"sell_price": "mean_sell_price"}, inplace=True)
    # data_df = data_df.merge(mean_sell_price_df["mean_sell_price"], on="id")
    # data_df["diff_sell_price"] = data_df["sell_price"] - data_df["mean_sell_price"]
    # data_df["div_sell_price"] = data_df["sell_price"] / data_df["mean_sell_price"]

    # time features
    data_df['date'] = pd.to_datetime(data_df['date'])
    # data_df['year'] = data_df['date'].dt.year.astype(np.int16)
    # data_df['quarter'] = data_df['date'].dt.quarter.astype(np.int8)
    # data_df['month'] = data_df['date'].dt.month.astype(np.int8)
    # data_df['week'] = data_df['date'].dt.week.astype(np.int8)
    # data_df['mday'] = data_df['date'].dt.day.astype(np.int8)
    # data_df['wday'] = data_df['date'].dt.dayofweek.astype(np.int8)
    # data_df['is_year_end'] = data_df['date'].dt.is_year_end.astype(np.int8)
    # data_df['is_year_start'] = data_df['date'].dt.is_year_start.astype.astype(np.int8)
    # data_df['is_quarter_end'] = data_df['date'].dt.is_quarter_end.astype(np.int8)
    # data_df['is_quarter_start'] = data_df['date'].is_quarter_start.astype(np.int8)
    # data_df['is_month_end'] = data_df['date'].dt.is_month_end.astype(np.int8)
    # data_df['is_month_start'] = data_df['date'].dt.is_month_start.astype(np.int8)
    # data_df["is_weekend"] = data_df["dayofweek"].isin([5, 6]).astype(np.int8)

    # black friday
    # black_friday = ["2011-11-25", "2012-11-23", "2013-11-29", "2014-11-28", "2015-11-27"]
    # data_df["black_friday"] = data_df["date"].isin(black_friday) * 1
    # data_df["black_friday"] = data_df["black_friday"].astype(np.int8)

    # lag特徴量によって欠損している部分を削除
    data_df.dropna(inplace = True)
    data_df = reduce_mem_usage(data_df)

    return data_df


# 交差検証はランダムサンプリングで疑似的に行う
def train_val_split(train_df):
    # train data
    X_train = train_df[train_df['part'] <= 'train']
    y_train = X_train['demand']

    # valid data
    # This is just a subsample of the training set, not a real validation set !
    print("\n[CHECK] This valid data is not a real valid data, just random sampling data from train data!")
    fake_valid_inds = np.random.choice(len(X_train), 1000000)
    print(fake_valid_inds)
    X_val = X_train.iloc[fake_valid_inds]
    y_val = X_val["demand"]

    del train_df
    gc.collect()

    return X_train, y_train, X_val, y_val


# # 交差検証は時系列に沿ったhold-out法で行う
# def data_split(data_df):
#     # train data
#     X_train = data_df[data_df['day'] <= 'd_1885']
#     y_train = X_train['demand']

#     # valid data
#     X_val = data_df[(data_df['day'] > 'd_1885') & (data_df['day'] <= 'd_1913')]
#     y_val = X_val['demand']

#     # stage1 eval data
#     test = data_df[(data_df['day'] > 'd_1913')]

#     del data_df
#     gc.collect()

#     return X_train, y_train, X_val, y_val, test


def train_lgb(X_train, y_train, X_val, y_val, features, date):
    # define random hyperparammeters
    params = {'boosting_type': 'gbdt',
              'metric': 'rmse',
              'objective': "poisson",
              'n_jobs': -1,
              'seed': 5046,
              'learning_rate': 0.075,
              'lambda_l2': 0.1,
              'sub_feature': 0.8,
              'sub_row': 0.75,
              'bagging_freq': 1}

    # datasetの作成
    train_set = lgb.Dataset(X_train[features], y_train)
    val_set = lgb.Dataset(X_val[features], y_val)

    # train/validation
    print("\n[START] training model ->")
    model = lgb.train(params, train_set, num_boost_round=2000,  # early_stopping_rounds=200,
                      valid_sets=[train_set, val_set], verbose_eval=100)

    # save model
    model_dir = "../model/{}".format(date[:8])
    os.makedirs(model_dir, exist_ok=True)
    model_name = "model_{}.pickle".format(date[9:])
    model_path = model_dir + model_name
    with open(model_path, mode='wb') as fp:
        pickle.dump(model, fp)

    val_pred = model.predict(X_val[features], num_iteration=model.best_iteration)
    val_RMSE_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
    print(f'val RMSE score: {val_RMSE_score}')

    return model


# 1日ずつ予測していく
def predict_lgb(model, test_df, features):
    train_day = test_df[test_df["part"]=="train"]["day"].unique()  # 特徴量作成のための部分
    stage1_day = test_df[test_df["part"]=="stage1"]["day"].unique()  # 実際に予測する部分
    print("[START] predict ->")

    for i in tqdm(range(28)):
        target_day = np.append(train_day, stage1_day[:i+1])  # 対象の指定(1日ずつ対象範囲を広げていく)
        partial_test = test_df[test_df["day"].isin(target_day)].copy()  # 該当するデータを抽出
        partial_test = feature_engineering(partial_test)
        partial_test = partial_test[partial_test["day"] == stage1_day[i]]  # 予測対象のみ抜粋(1日)
        partial_pred = model.predict(partial_test[features])
        print(partial_pred)
        print(np.unique(partial_pred))
        test_df[test_df["day"]==stage1_day[i]]['demand'] = 1.02 * partial_pred
        break

    return test_df


def result_submission(test_df, submission_df, date):
    output_dir = "../output/{}".format(date[:8])
    os.makedirs(output_dir, exist_ok=True)
    submission_name = "submission_{}.csv".format(date[9:])
    submission_path = output_dir + submission_name
    

    pred = test_df[['id', 'date', 'demand']]
    pred = pd.pivot(pred, index='id', columns='date', values='demand').reset_index()
    pred.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    eval_rows = [row for row in submission_df['id'] if 'evaluation' in row]
    eval = submission_df[submission_df['id'].isin(eval_rows)]

    valid = submission_df[['id']].merge(pred, on='id')
    final = pd.concat([valid, eval])
    final.to_csv(submission_path, index=False)
    print("\ncompleted process.")

In [2]:


# 変更パラメータ
num_train_data = 500  # 421 = 365 + 28*2

# read data
t1 = time.time()
date = datetime.datetime.today().strftime("%Y%m%d_%H%M%S")
calendar_df, sell_prices_df, train_df, submission_df = read_data()

# preprocessing
data_df = melt_and_merge(calendar_df, sell_prices_df, train_df, submission_df, num_train_data)
data_df = encode_categorical(data_df)

# data split
train_df, test_df = data_split(data_df)

# define list of features
default_features = ['item_id', 'dept_id', 'store_id','cat_id', 'state_id', 'event_name_1', 'sell_price']
demand_features = ['lag7', 'lag28', 'rmean_lag7_7', 'rmean_lag7_28', 'rmean_lag28_7', 'rmean_lag28_28']
# price_features = ['sell_price_lag1', 'sell_price_lag7', 'sell_price_lag28', "diff_sell_price", "div_sell_price"]
# time_features = ["year", "month", "week", "quarter", "mday", "wday", "black_friday"]
features = default_features + demand_features  # + price_features + time_features
print("N_features: {}\n".format(len(features)))

# train
with open("../model/20200408model_115157.pickle", mode='rb') as fp:
    model = pickle.load(fp)

Reading files...
Calendar: (1969, 14)
Sell prices: (6841121, 4)
Sales train validation: (30490, 1919)
Submission: (60980, 29)

trainは直近１年間のデータのみ使用
Sales train validation(remain only one year): (30490, 506)
train_data: d_1414 ~ d_1913 -> 500
[STAGE1] eval_data: d_1914 ~ d_1941 -> 28
[STAGE2] eval_data: d_1942 ~ d_1969 -> 28
[CHECK] Remove the stage2 eval data
N_features: 13



In [8]:
train_day = test_df[test_df["part"]=="train"]["day"].unique()  # 特徴量作成のための部分
stage1_day = test_df[test_df["part"]=="stage1"]["day"].unique()  # 実際に予測する部分

In [14]:
# predict
test_df2 = test_df.copy()
for i in range(28):
    target_day = np.append(train_day, stage1_day[:i+1])  # 対象の指定(1日ずつ対象範囲を広げていく)
    partial_test = test_df2[test_df2["day"].isin(target_day)].copy()  # 該当するデータを抽出
    partial_test = feature_engineering(partial_test)
    partial_test = partial_test[parge1_day[i]test["day"] == stage1_day[i]]  # 予測対象のみ抜粋(1日)
    partial_pred = model.predict(partial_test[features])
    print(partial_pred)
    print(len(partial_pred))
    test_df2[test_df2["day"]==stage1_day[i]]['demand'] = 1.02 * partial_pred
    break

[0.95573836 0.24375241 0.68493054 ... 1.08343165 1.00747958 1.27688968]
30490


In [16]:
test_df2[test_df2["day"]==stage1_day[i]]['demand']

30490

In [18]:
test_df2.loc[test_df2["day"]==stage1_day[0], "demand"] = partial_pred

In [19]:
test_df2[test_df2["day"]==stage1_day[i]]['demand']

15245000     0.955738
15245001     0.243752
15245002     0.684931
15245003     1.940756
15245004     1.039538
15245005     0.990465
15245006     0.409134
15245007     7.158406
15245008     0.761677
15245009     0.541631
15245010     0.143886
15245011     0.201172
15245012     0.499007
15245013     1.748715
15245014     3.355362
15245015     6.314783
15245016     1.101404
15245017     0.049151
15245018     8.055341
15245019     0.262996
15245020     0.639433
15245021     0.449032
15245022     1.264680
15245023     0.121635
15245024     0.533376
15245025     0.253089
15245026     0.326054
15245027     0.706595
15245028     1.539399
15245029     4.262160
              ...    
15275460     0.426455
15275461     0.321897
15275462     7.271266
15275463     1.991804
15275464     0.508307
15275465     0.626882
15275466     6.592206
15275467     1.970911
15275468     0.768598
15275469     5.002996
15275470     0.533904
15275471     2.347900
15275472     0.723049
15275473    13.819510
15275474  